# **Importing signal**

## Signal info

In [22]:
import mne
import numpy as np

file = "firsttest.edf"
eeg = mne.io.read_raw_edf(f"C:/Users/migue/OneDrive/Ambiente de Trabalho/Tese/EEG Data/{file}")

print("\n", eeg.info)

ch = eeg.info["ch_names"] # Channel labels
fs = eeg.info["sfreq"]

print("\nChannel labels: ", ch)

Extracting EDF parameters from C:\Users\migue\OneDrive\Ambiente de Trabalho\Tese\EEG Data\firsttest.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

 <Info | 7 non-empty values
 bads: []
 ch_names: EEG TP7'', EEG P7'', EEG PO7'', EEG CP5'', EEG P5'', EEG PO5'', ...
 chs: 36 EEG
 custom_ref_applied: False
 highpass: 0.1 Hz
 lowpass: 269.0 Hz
 meas_date: 2024-02-16 12:39:22 UTC
 nchan: 36
 projs: []
 sfreq: 1024.0 Hz
>

Channel labels:  ["EEG TP7''", "EEG P7''", "EEG PO7''", "EEG CP5''", "EEG P5''", "EEG PO5''", "EEG C3''", "EEG CP3''", "EEG P3''", "EEG PO3''", "EEG FP1''", "EEG CP1''", "EEG P1''", "EEG O1''", "EEG Cz''", "EEG CPz''", "EEG Pz''", "EEG POz''", "EEG Oz''", "EEG FP2''", "EEG CP2''", "EEG P2''", "EEG O2''", "EEG C4''", "EEG CP4''", "EEG P4''", "EEG PO4''", "EEG CP6''", "EEG P6''", "EEG PO6''", "EEG TP8''", "EEG P8''", "EEG PO8''", 'dc01+', 'dc02+', 'MKR+']


## Isolating the LDR channel

In [23]:
ldr_ch = eeg.pick_channels(['dc01+']).get_data() 
if (np.mean(ldr_ch) < 0): # If the polarity was inverted
    ldr_ch = ldr_ch * -1
''' 
The bright square (more light) will decrease the resistance of the LDR, thus producing more voltage
aproximating 2.4V if the polarity of the component was right or -2.4V if the polarity was inverted

The dark square will increase the resistance to the MOhm range and cause the signal to be almost 0.
This is because all the voltage will be sucked by the first component (LDR) of the voltage divider.
'''

time = np.linspace(0, np.size(ldr_ch,1)/fs, np.size(ldr_ch,1))/60 # Time vector of the acquisition

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize = (400,20))
plt.plot(time, ldr_ch[:][0]);
plt.title('LDR Channel', fontsize = 100, fontweight = 'bold')
plt.xlabel('Time (min)', fontsize = 85, fontweight = 'bold')
plt.ylabel('Voltage (V)', fontsize = 85, fontweight = 'bold')
plt.xticks(np.arange(0, np.max(time), 0.05), fontsize = 50)
plt.yticks(np.arange(np.min(ldr_ch), np.max(ldr_ch), 0.1), fontsize = 50)
plt.show()

## Filtering the LDR channel

In [24]:
import scipy
filt_ldr_ch = scipy.signal.medfilt(ldr_ch[:][0],201) 
# A 201 points median filter seems to be enough to smooth the signal without causing phase shifts
# The filter applies a zero pad automatically at the edges

figure(figsize = (400,20))
plt.plot(time, filt_ldr_ch);
plt.title('Filtered LDR Channel', fontsize = 100, fontweight = 'bold')
plt.xlabel('Time (min)', fontsize = 85, fontweight = 'bold')
plt.ylabel('Voltage (V)', fontsize = 85, fontweight = 'bold')
plt.xticks(np.arange(0, np.max(time), 0.05), fontsize = 50)
plt.yticks(np.arange(np.min(ldr_ch), np.max(ldr_ch), 0.1), fontsize = 50)
plt.show()

# **Segmenting LDR channel**

In [25]:
def segmentation(pht_resistor, th):
    
    last_low = True
    last_high =False
    
    falling_edges = []
    rising_edges = []

    for count, val in enumerate(pht_resistor):

        if val >= th: # Possible rising edge
            if last_low: # Found rising edge
                rising_edges.append(count)
            
            last_low = False
            last_high = True
        
        elif val <= th: # Possible falling edge
            if last_high: # Found falling edge
                falling_edges.append(count)

            last_low = True
            last_high = False

    return rising_edges, falling_edges